In [16]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

#tb.loc[tb['Instrument Classification (ISO 10962 CFI code)'].isin(['JFTXFP'])]
#tb.plot.hist(tb.groupby(['Trading platform']).Price.agg(['mean', max]))
#tb.describe()
#print(tb.shape)

df = pd.read_feather('C:/Users/asus/Documents/rts27.feather')

#Columns deemed unneccessary for this task
df2 = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)


In [36]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets, linear_model
from sklearn.decomposition import TruncatedSVD


from datetime import datetime

#Columns deemed unneccessary for this task
#df = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)
df = df2


#Filters the code to FX forwards only
#df = df.loc[(df['source_instrument_desc']).astype(str).str.contains("Forward")]
df = df.loc[(df['instrument_classification']).astype(str).str.contains("JF") | (df['source_instrument_desc']).astype(str).str.contains("Forward")]
#Drops the columns of which half of the fields have no data.
#df = df.dropna(axis=1, thresh=2000000)

        
# Select target
y = df['date_created']
# To keep things simple, we'll use only numerical predictors
dropped_df = df.drop(['date_created', 'instrument_identifier'], axis=1)
X = dropped_df


# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

categorical_cols = [col for col in X.columns if X[col].dtype in ['object']]

# Select numerical columns
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant',missing_values=np.nan, fill_value=0)

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', missing_values= None, fill_value='missing_value')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

sc = StandardScaler()

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = linear_model.LinearRegression()

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('scale', StandardScaler(with_mean=False)),
                              ('reduce_dims', TruncatedSVD(n_components=5)),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Plot outputs
print(preds)
#plt.plot(X_valid, preds, color='blue', linewidth=3)

[1.55413620e+18 1.55365074e+18 1.54403796e+18 ... 1.54360285e+18
 1.55697413e+18 1.54826506e+18]
